In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import preprocessing
import time

In [2]:
tic=time.time()
df = pd.read_csv('/Users/eduardorodriguez/Desktop/TFM/DATA/banksim/banksim.csv')

def fx(x):
    return str(x[1:-1])

def fx2(x):
    if x == "'U'":
        return 7
    else:
        return int(x[1:-1])


df['category'] = df.category.apply(fx)
df['gender'] = df.gender.apply(fx)
df['age'] = df.age.apply(fx2)

le = preprocessing.LabelEncoder()
le.fit(list(set(df['category'])))
df['category'] = le.transform(df['category'])

le.fit(list(set(df['age'])))
df['age'] = le.transform(df['age'])

le.fit(list(set(df['gender'])))
df['gender'] = le.transform(df['gender'])

df_data = df
#X = df.loc[(df.category == 'es_sportsandtoys') | (df.category == 'es_fashion') | (df.category == 'es_travel') | (df.category == 'es_home') |(df.category == 'es_health') | (df.category == 'es_leisure') | df.category == 'es_tech') | (df.category == 'es_otherservices') | (df.category == 'es_barsandrestaurants') |(df.category == 'es_hyper') | (df.category == 'es_wellnessandbeauty')]
y_true = df_data['fraud']

df = df.drop(['customer', 'merchant', 'zipMerchant', 'zipcodeOri'], axis = 1)
df = df.rename(index=str, columns={"fraud": "Class"})
df_data = df_data.drop(['customer', 'merchant', 'zipMerchant', 'zipcodeOri'], axis = 1)
full_data = df

In [3]:
ones = full_data[(full_data.Class == 1)]
zeroes = full_data[(full_data.Class == 0)][:len(ones)*4]

X = pd.concat([ones, zeroes])
y_true = X['Class']
full_data = X
full_data.shape

(36000, 6)

In [4]:
full_data=full_data.sample(frac=1)#randomize the whole dataset
full_features=full_data.drop(["step","Class"],axis=1)
full_labels=pd.DataFrame(full_data[["Class"]])
full_features_array=full_features.values
full_labels_array=full_labels.values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(full_data, y_true, test_size=0.3, random_state=42)

In [6]:
def train(X,clf,ensembleSize=5,sampleSize=10000):
    mdlLst=[]
    for n in range(ensembleSize):
        X=df_data.sample(sampleSize)
        clf.fit(X)
        mdlLst.append(clf)
    
    return mdlLst

In [7]:
## Not valid for LOF
def predict(X,mdlLst):
    y_pred=np.zeros(X.shape[0])
    for clf in mdlLst:
        y_pred=np.add(y_pred,clf.decision_function(X).reshape(X.shape[0],))
    y_pred=(y_pred*1.0)/len(mdlLst)
    return y_pred

In [ ]:
acuracy = []
precision = []
recall = []
f1 = []
timekm = []
n = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
for i in n:
    print('N_Clusters = ' + str(i) + '\\\\' + '\n' + '\\\\')
    
    #k_means_classification --> k_means_clustering, confsion_matrix, reassigning
    kmeans = KMeans(n_clusters=i, random_state=42,n_jobs=-1).fit(X_train)

    X_test_clusters=kmeans.predict(X_test)
    X_test_clusters_centers=kmeans.cluster_centers_
    dist = [np.linalg.norm(x-y) for x,y in zip(X_test.as_matrix(),X_test_clusters_centers[X_test_clusters])]

    km_y_pred=np.array(dist)
    km_y_pred[dist>=np.percentile(dist,95)]=1
    km_y_pred[dist<np.percentile(dist,95)]=0
    
    X_test_clusters=kmeans.predict(X_test)
    X_test_clusters_centers=kmeans.cluster_centers_
    dist = [np.linalg.norm(x-y) for x,y in zip(X_test.as_matrix(),X_test_clusters_centers[X_test_clusters])]

    km_y_pred=np.array(dist)
    km_y_pred[dist>=np.percentile(dist,95)]=1
    km_y_pred[dist<np.percentile(dist,95)]=0
    
    km_cm=confusion_matrix(y_test, km_y_pred)
    #scoring kmeans
    kmeans_accuracy_score=accuracy_score(y_test, km_y_pred)
    kmeans_precison_score=precision_score(y_test, km_y_pred)
    kmeans_recall_score=recall_score(y_test, km_y_pred)
    kmeans_f1_score=f1_score(y_test, km_y_pred)
    
    #printing
    print("K-Means")
    print("Confusion Matrix\\\\")
    print("tn =",km_cm[0][0],"fp =",km_cm[0][1], end='')
    print('\\\\')
    print("fn =",km_cm[1][0],"tp =",km_cm[1][1], end='')
    print('\\\\')
    print("Scores", end='')
    print('\\\\')
    print("Accuracy -->",kmeans_accuracy_score, end='')
    print('\\\\')
    print("Precison -->",kmeans_precison_score, end='')
    print('\\\\')
    print("Recall -->",kmeans_recall_score, end='')    
    print('\\\\')
    print("F1 -->",kmeans_f1_score)
    print('\\\\')
    print('\\\\')
    print('\\\\')

    #time elapsed
    toc=time.time()
    elapsedtime=toc-tic
    print("Time Taken : "+str(elapsedtime)+"seconds\n\n")
    
    acuracy += [kmeans_accuracy_score,]
    precision += [kmeans_precison_score,]
    recall += [kmeans_recall_score,]
    f1 += [kmeans_f1_score,]
    timekm += [elapsedtime,]

N_Clusters = 2\\
\\


/Users/eduardorodriguez/anaconda2/envs/TFM/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  from ipykernel import kernelapp as app
/Users/eduardorodriguez/anaconda2/envs/TFM/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


K-Means
Confusion Matrix\\
tn = 8188 fp = 434\\
fn = 2072 tp = 106\\
Scores\\
Accuracy --> 0.767962962962963\\
Precison --> 0.1962962962962963\\
Recall --> 0.048668503213957756\\
F1 --> 0.07799852832965415
\\
\\
\\
Time Taken : 5.8730309009552seconds


N_Clusters = 3\\
\\
K-Means
Confusion Matrix\\
tn = 8190 fp = 432\\
fn = 2070 tp = 108\\
Scores\\
Accuracy --> 0.7683333333333333\\
Precison --> 0.2\\
Recall --> 0.049586776859504134\\
F1 --> 0.07947019867549669
\\
\\
\\
Time Taken : 6.741516828536987seconds


N_Clusters = 4\\
\\
K-Means
Confusion Matrix\\
tn = 8183 fp = 439\\
fn = 2077 tp = 101\\
Scores\\
Accuracy --> 0.7670370370370371\\
Precison --> 0.18703703703703703\\
Recall --> 0.046372819100091826\\
F1 --> 0.07431935246504782
\\
\\
\\
Time Taken : 7.716099739074707seconds


N_Clusters = 5\\
\\
K-Means
Confusion Matrix\\
tn = 8186 fp = 436\\
fn = 2074 tp = 104\\
Scores\\
Accuracy --> 0.7675925925925926\\
Precison --> 0.1925925925925926\\
Recall --> 0.047750229568411386\\
F1 --> 0.

In [ ]:
acuracy = list(acuracy)

In [ ]:
len(acuracy)

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
 
y_pos = n[:-3]
performance = list(map(lambda x: x - min(acuracy) + 0.00001, acuracy))
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, n[:-3])
plt.ylabel('Percentage from 0.939')
plt.title('Acuracy')
 
plt.show()

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
 
y_pos = n[:-3]
performance = list(map(lambda x: x - min(recall) + 0.00001, recall))
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, n[:-3])
plt.ylabel('Percentage')
plt.title('Recall')
 
plt.show()

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
 
y_pos = n[:-3]
performance = list(map(lambda x: x - min(f1) + 0.00001, f1))
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, n[:-3])
plt.ylabel('Percentage')
plt.title('F1')
 
plt.show()